# **Installing the dependencies**

In [ ]:
!pip install transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 39.0 MB/s eta 0:00:00


In [ ]:
!pip install -U SentencePiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.9 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 3.7 MB/s eta 0:00:00


In [ ]:
CUDA_LAUNCH_BLOCKING=1 # to stop cuda blocking

# **Connecting google colab to drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Importing the dependencies**

In [ ]:
import json
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification, T5Tokenizer, T5ForSequenceClassification,TrainingArguments, Trainer

## Data format

2 json files - `surprise.data` contains utterances, `surprise.solution` contains corresponding intents

Format of `surprise.data`

```json
{"indoml_id": "surprise|11109", "id": "11109", "utt": "Can I make a reservation at Buffalo Wild Wings?"}
{"indoml_id": "surprise|11051", "id": "11051", "utt": "Can I book a table for tonight at Bella Vita?"}
....
```

Format of `surprise.solution` (contains 150 labels)

```json
{"indoml_id": "surprise|11109", "intent": "accept reservations"}
{"indoml_id": "surprise|11051", "intent": "accept reservations"}
....
```


# **Loading the data and solution of the surprise dataset**

In [ ]:

# Load the data from the JSON files
with open('/content/drive/MyDrive/intent_classification_It_bombay/surprise_data/surprise.data', 'r') as data_file:
    data = [json.loads(line) for line in data_file] # converting string to object using json.loads

with open('/content/drive/MyDrive/intent_classification_It_bombay/surprise_data/surprise.solution', 'r') as solution_file:
    solutions = [json.loads(line) for line in solution_file] # converting string to object using json.loads

In [ ]:
print(data[0]) # data of the surpirse dataset having id and the utterances
print(solutions[0]) # labels of the surprise data containing id and the utterances

{'indoml_id': 'surprise|11109', 'id': '11109', 'utt': 'Can I make a reservation at Buffalo Wild Wings?'}
{'indoml_id': 'surprise|11109', 'intent': 'accept reservations'}


In [ ]:
utt_list=[]
for data_1 in data:
    utt=data_1['utt']
    utt_list.append(utt)
utt_list

['Can I make a reservation at Buffalo Wild Wings?',
 'Can I book a table for tonight at Bella Vita?',
 'Does the Cheesecake Factory take reservations?',
 'Do they take reservations at Outback Steakhouse in Las Vegas?',
 "does chili's take reservations?",
 'Can I make a reservation at The French Laundry in Yountville?',
 'Can I check availability and make a reservation online?',
 "Can you tell me if Carrabba's in Houston accepts reservations",
 'Do you have any availability for next weekend at the Brasserie?',
 'Does Buffalo Wild Wings take reservations for sports games',
 'Is it possible to reserve a table in advance at this restaurant?',
 'Does Olive Garden in San Francisco take reservations?',
 "Where can I find reservation information for Applebee's?",
 'Can I reserve a table at Alinea in Chicago?',
 'Does the Pizza Parlor accept reservations for parties of five or more?',
 'Why did my brokerage account suddenly get locked?',
 "I've tried to log into my Chase account, but it's still

In [ ]:
# Create a dictionary to map indoml_id to intents
intent_map = {item['indoml_id']: item['intent'] for item in solutions}

# Split data into train and test sets (2:1) stratified by intent
indoml_ids = [item['indoml_id'] for item in data]
intents = [intent_map[indoml_id] for indoml_id in indoml_ids]
"""utterances(features) for the dataset"""
utt = [item['utt'] for item in data]

num_classes = len(set(intents))
print("Number of classes")
print(num_classes)

"""Splitting the dataset into train and test set"""
train_data, test_data, train_labels, test_labels = train_test_split(
    utt, intents, test_size=0.20, random_state=42, stratify=intents
)

Number of classes
150


In [ ]:
train_labels[0:5], len(train_labels) # these are basically our intents corrosponding to the utterances for trainin g data(5%) of the data

(['insurance',
  'car rental',
  'international visa',
  'definition',
  'report fraud'],
 1798)

In [ ]:
test_labels[0:5], len(test_labels) # these are basically our intents corrosponding to the utterances for testing data(5%) of the data

(['car rental', 'repeat', 'confirm reservation', 'todo list', 'gas type'], 450)

#**label encoding- converting labels to numeric values**

In [ ]:
labels_list=[]
for label in solutions:
    labels_list.append(label['intent'])
unique_labels_list=[]
for x in labels_list:
    if x not in unique_labels_list:
        unique_labels_list.append(x)
# unique_labels_list, len(unique_labels_list)

label2id={}
id2label={}
for i, label in enumerate(unique_labels_list):
    label2id[label]=i
    id2label[i]=label

In [ ]:
id2label

In [ ]:
print(list(id2label.items())[:5])
print('\n')
print(list(label2id.items())[:5])


[(0, 'accept reservations'), (1, 'account blocked'), (2, 'alarm'), (3, 'application status'), (4, 'apr')]


[('accept reservations', 0), ('account blocked', 1), ('alarm', 2), ('application status', 3), ('apr', 4)]


# **Loading the pretrained intent classification Alexa XLMRoberta model and the tokenizer**

**Here first we hve trained a model on ntu adl intent dataset containing same number of intents,**

**Then based on the best checkpoint(on the basis of best Validation Score) after training roberta large on that ntu adl dataset obtained from there will be used for fine tuning the model on our surprise dataset**.
this is basically the first stage fine tuning where we are picking pretrained roberta large model on Sequence classification task and fine tuning that on ntu adl intent dataset.

Below is the link of that ntu adl intent dataset

 [https://huggingface.co/datasets/xjlulu/ntu_adl_intent](https://)


*From here after tuning the model on that checkpoint we will saved the model weigths(checkpoints) and best model weights on the basis of validation score will be used for predicting the labels of massive test dataset(this is basically the 2nd stage fine tuning)*

In [ ]:

from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
# 2nd stage fine tuning
model_name="/content/drive/MyDrive/roberta_large_2nd_stage_training_epoch_11/epoch_21" # pretrained model
tokenizer=RobertaTokenizer.from_pretrained(model_name) # loading tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Initialize the model
num_classes=150
# defining model
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=num_classes, ignore_mismatched_sizes=True)
"""since model have its own id2label mapping and vice versa so, converting them from our id2label and label2id mapping as defined earlier"""
model.label2id=label2id
model.id2label=id2label


In [ ]:
# Tokenize the input training data
train_encodings = tokenizer(
    train_data,
    truncation=True,
    padding=True,
    max_length=64,
    return_tensors='pt'    #return type is pytorch tensor
)
#Tokenizing the input testing data
test_encodings = tokenizer(
    test_data,
    truncation=True,
    padding=True,
    max_length=64,
    return_tensors='pt'
)

In [ ]:
"""Convert labels to numeric values from label2id mapping"""
train_int_labels = [label2id[label] for label in train_labels]
test_int_labels = [label2id[label] for label in test_labels]
test_int_labels[0:5]

[117, 139, 67, 39, 18]

# **making pytorch datasets**

In [ ]:
# Create PyTorch datasets
class IntentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IntentDataset(train_encodings, train_int_labels)
test_dataset = IntentDataset(test_encodings, test_int_labels)

In [ ]:
from torch.utils.data import DataLoader
batch_size = 64  # Adjust the batch size as needed
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True  # You can shuffle your data for randomness during training
)
eval_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False # You can shuffle your data for randomness during training
)


In [ ]:
# Training arguments
# previous_saved_dir='./intent_classification__'
saved_dir='/content/drive/MyDrive/intent_classification_It_bombay/trained_model_10_checkpoints/'
training_args = TrainingArguments(
    output_dir=saved_dir,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    seed=42,
    # weight-decay=0.01
    # gradient_accumulation_steps=40,
    evaluation_strategy="steps", # use 'epoch' for evaluating every epoch
    logging_steps=10,
    eval_steps=10,
    save_total_limit=5,
    save_steps=60,
    learning_rate=7e-6,
    # warmup_steps=400,
    # weight_decay=0.10,
    # adam_epsilon=1e-7,
    # warmup_steps=400,
    num_train_epochs=40,
    logging_dir='./logs',
)

In [ ]:
import os
import torch
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import TrainingArguments
from sklearn.model_selection import KFold
from tqdm import tqdm

# Define your model, tokenizer, and other necessary components here
# Make sure your model is moved to the GPU if available

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the learning rate as a float (e.g., 7e-6)
learning_rate = 1e-5

# Define the number of training epochs as an integer (e.g., 40)
num_train_epochs = 40

# Create the optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Define the number of training steps (you may need to adjust this based on your dataset)
total_train_steps = len(train_dataloader) * num_train_epochs

# Define the number of warmup steps (e.g., 10% of the total training steps)
num_warmup_steps = int(0.1 * total_train_steps)

# Create a learning rate scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=total_train_steps,
)

# Define the output directory for saving the model
saved_dir="/content/drive/MyDrive/intent_classification_It_bombay/trained_model_final_2nd_stage_less_lr" # this is the directory where all weights will get saved after training the model for 2nd stage fine tuning
# Training loop with tqdm progress bar
for epoch in range(num_train_epochs):
    model.train()
    train_loss = 0.0

    progress_bar = tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f"Epoch {epoch}")

    for step, batch in progress_bar:
        # Move data to the GPU
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Update the learning rate
        lr_scheduler.step()

        # Accumulate the total loss
        train_loss += loss.item()

        # Log loss and other metrics if needed
        if step % training_args.logging_steps == 0:
            avg_loss = train_loss / (step + 1)
            progress_bar.set_postfix(loss=avg_loss)

    # Save the model checkpoint at the end of each epoch
    checkpoint_dir = os.path.join(saved_dir, f"epoch_{epoch}")
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Save model weights
    model.save_pretrained(checkpoint_dir)

    # Save optimizer state
    # torch.save(optimizer.state_dict(), os.path.join(checkpoint_dir, "optimizer.pt"))

    # # Save learning rate scheduler state
    # torch.save(lr_scheduler.state_dict(), os.path.join(checkpoint_dir, "scheduler.pt"))

    # Save RNG state (if needed)
    # torch.save(torch.get_rng_state(), os.path.join(checkpoint_dir, "rng_state.pth"))

    # Save config.json (if needed)
    model.config.save_pretrained(checkpoint_dir)

    # Save tokenizer (if needed)
    tokenizer.save_pretrained(checkpoint_dir)

    # Evaluation loop with tqdm progress bar
    model.eval()
    eval_loss = 0.0

    progress_bar_eval = tqdm(enumerate(eval_dataloader), total=len(eval_dataloader), desc=f"Evaluation")

    for step, batch in progress_bar_eval:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
            loss = outputs.loss
            eval_loss += loss.item()

    avg_eval_loss = eval_loss / len(eval_dataloader)
    print(f"Epoch {epoch}: Evaluation Loss: {avg_eval_loss}")
    print(f'learning rate is : {learning_rate}')

    # if (epoch + 1) % 3 == 0:
    #     learning_rate *= 0.75
    #     for param_group in optimizer.param_groups:
    #         param_group['lr'] = learning_rate


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Evaluation: 100%|██████████| 8/8 [00:02<00:00,  2.91it/s]


Epoch 0: Evaluation Loss: 9.033305525779724
learning rate is : 1e-05


Evaluation: 100%|██████████| 8/8 [00:02<00:00,  2.85it/s]


Epoch 1: Evaluation Loss: 4.901621520519257
learning rate is : 1e-05


Evaluation: 100%|██████████| 8/8 [00:02<00:00,  2.78it/s]


...

Epoch 18: Evaluation Loss: 0.25928995572030544
learning rate is : 1e-05


Evaluation: 100%|██████████| 8/8 [00:03<00:00,  2.61it/s]


Epoch 19: Evaluation Loss: 0.28044209629297256
learning rate is : 1e-05


Epoch 20:  48%|████▊     | 14/29 [00:26<00:28,  1.89s/it, loss=0.0434]


KeyboardInterrupt: ignored

# **prediction on the test set(whose labels is available)**

In [ ]:
# raw_predictions = model.predict(test_dataset)
# predicted_labels = np.argmax(raw_predictions.predictions)
# predicted_labels, raw_predictions
import numpy as np
import torch
model_name="/content/drive/MyDrive/intent_classification_It_bombay/trained_model_final_3rd_stage_less_lr/epoch_18" # Taking the bestcheckpoint from above training for evaluating the performance of the model on the validation dataset
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=num_classes, ignore_mismatched_sizes=True)

# Put your test data into a DataLoader or batch format if it's not already
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model.to(device)
# Set the model to evaluation mode
model.eval()

predicted_labels = []
raw_predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        # Move batch to the GPU if available
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)
        logits = outputs.logits

        # Get predicted labels (argmax)
        batch_predictions = np.argmax(logits.cpu().numpy(), axis=1)

        # Append batch predictions to the list
        predicted_labels.extend(batch_predictions)

        # Append raw logits if needed
        raw_predictions.extend(logits.cpu().numpy())

# Convert the results to numpy arrays
predicted_labels = np.array(predicted_labels)
raw_predictions = np.array(raw_predictions)

# Now, you have predicted labels and raw logits.


In [ ]:
predicted_labels, raw_predictions

(array([117, 139,  67,  39,  18, 126,  13,  95,  46,  83,  76, 123, 145,
         32, 112, 136, 106,   2,  36,  38,  55, 149, 106,  45,  92, 108,
         50,  46,  93,  81,  90,  31,  24,  10,  27, 146, 129,  93,  65,
         70,  17, 121,  20,  55,  53,  11,   8, 110, 128, 116,  61,   1,
         88,   7, 141,  62, 137, 105,  68,  64,  27,  66,  47,  76,  54,
         91,  48, 132,  25, 137,  84,  32,  22, 107,  69,  44,  16, 130,
        114, 103,  84,  72,  39, 110, 142,  79,  21, 135, 100,  43, 133,
         51, 107,  25,  74,  95,  87,  55,  58,  21, 106,   6,  42,   9,
         37, 140, 131, 146,  64, 103,  61,  11,  15, 113, 137, 100,  49,
         81,  96,  89, 115, 105, 139,  68, 139,  19,  56,  89,  15, 147,
        132,  17,  38, 147,   5,  13,  12,  65,  86, 129, 146,  49,  80,
         20,  93, 116, 119, 121,  73,  69,  54,  51,  51,  36,  31, 138,
         73, 101,  63,  65, 136, 126,  16, 102,  24,  58,  52,  72,  79,
         52, 134,  75, 147,  91,  56,  83,  44,  82

In [ ]:

# # Predict on the test set
# raw_predictions = trainer.predict(test_dataset)
# predicted_labels = np.argmax(raw_predictions.predictions, axis=1)

# # Convert labels back to original intents
# # predicted_intents = [list(label_map.keys())[list(label_map.values()).index(label)] for label in predicted_labels]
# predicted_intents=[id2label[label] for label in predicted_labels]

# # Print one sample prediction
# sample_idx = random.randint(0, len(test_data) - 1)
# sample_text = test_data[sample_idx]
# sample_intent = predicted_intents[sample_idx]

# print(f"Sample Text: {sample_text}")
# print(f"Predicted Intent: {sample_intent}")
import random
import numpy as np

# Put your test data into a DataLoader or batch format if it's not already
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Set the model to evaluation mode
model.eval()

predicted_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        # Move batch to the GPU if available
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)
        logits = outputs.logits

        # Get predicted labels (argmax)
        batch_predictions = np.argmax(logits.cpu().numpy(), axis=1)

        # Append batch predictions to the list
        predicted_labels.extend(batch_predictions)

# Convert the results to numpy arrays
predicted_labels = np.array(predicted_labels)

# Convert labels back to original intents
predicted_intents = [id2label[label] for label in predicted_labels]

# Print one sample prediction
sample_idx = random.randint(0, len(test_data) - 1)
sample_text = test_data[sample_idx]
sample_intent = predicted_intents[sample_idx]

print(f"Sample Text: {sample_text}")
print(f"Predicted Intent: {sample_intent}")


Sample Text: How much will it cost to take an Uber from LAX to downtown LA?
Predicted Intent: uber


# **metrics(accuracy criterias)**

In [ ]:
# Calculate accuracy, precision, recall, and F1-score
report = classification_report(test_int_labels, predicted_labels, target_names=list(label2id.keys()), output_dict=True)
print(report)
print('\n')
accuracy = report['accuracy']
precision = report['macro avg']['precision']
recall = report['macro avg']['recall']
f1_score = report['macro avg']['f1-score']

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")

{'jump start': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 3}, 'schedule meeting': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 3}, 'bill due': {'precision': 0.75, 'recall': 1.0, 'f1-score': 0.8571428571428571, 'support': 3}, 'change language': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 3}, 'pin change': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 3}, 'whisper mode': {'precision': 1.0, 'recall': 0.6666666666666666, 'f1-score': 0.8, 'support': 3}, 'date': {'precision': 1.0, 'recall': 0.6666666666666666, 'f1-score': 0.8, 'support': 3}, 'what is your name': {'precision': 1.0, 'recall': 0.6666666666666666, 'f1-score': 0.8, 'support': 3}, 'translate': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 3}, 'sync device': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 3}, 'insurance change': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 3}, 'who made you': {'precision': 1.0, 'reca

In [ ]:
report

{'jump start': {'precision': 1.0,
  'recall': 1.0,
  'f1-score': 1.0,
  'support': 3},
 'schedule meeting': {'precision': 1.0,
  'recall': 1.0,
  'f1-score': 1.0,
  'support': 3},
 'bill due': {'precision': 0.75,
  'recall': 1.0,
  'f1-score': 0.8571428571428571,
  'support': 3},
 'change language': {'precision': 1.0,
  'recall': 1.0,
  'f1-score': 1.0,
  'support': 3},
 'pin change': {'precision': 1.0,
  'recall': 1.0,
  'f1-score': 1.0,
  'support': 3},
 'whisper mode': {'precision': 1.0,
  'recall': 0.6666666666666666,
  'f1-score': 0.8,
  'support': 3},
 'date': {'precision': 1.0,
  'recall': 0.6666666666666666,
  'f1-score': 0.8,
  'support': 3},
 'what is your name': {'precision': 1.0,
  'recall': 0.6666666666666666,
  'f1-score': 0.8,
  'support': 3},
 'translate': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 3},
 'sync device': {'precision': 1.0,
  'recall': 1.0,
  'f1-score': 1.0,
  'support': 3},
 'insurance change': {'precision': 1.0,
  'recall': 1.0,
  'f1-

In [ ]:
del report['accuracy']

In [ ]:
import pandas as pd

# Initialize an empty list to store the data as dictionaries
data_list = []

# Iterate through the data_dict and convert each entry into a dictionary
for label, metrics in report.items():
    entry = {'label': label}
    print(entry)
    entry.update(metrics)

    data_list.append(entry)

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data_list)

# Print the resulting DataFrame
print(df)


{'label': 'reminder'}
{'label': 'book hotel'}
{'label': 'translate'}
{'label': 'international fees'}
{'label': 'who do you work for'}
{'label': 'routing'}
{'label': 'recipe'}
{'label': 'redeem rewards'}
{'label': 'change accent'}
{'label': 'pto request'}
{'label': 'pto used'}
{'label': 'calculator'}
{'label': 'are you a bot'}
{'label': 'pto balance'}
{'label': 'rewards balance'}
{'label': 'improve credit score'}
{'label': 'traffic'}
{'label': 'roll dice'}
{'label': 'replacement card duration'}
{'label': 'time'}
{'label': 'pin change'}
{'label': 'meal suggestion'}
{'label': 'rollover 401k'}
{'label': 'taxes'}
{'label': 'change ai name'}
{'label': 'credit score'}
{'label': 'carry on'}
{'label': 'ingredients list'}
{'label': 'expiration date'}
{'label': 'order checks'}
{'label': 'whisper mode'}
{'label': 'play music'}
{'label': 'restaurant reviews'}
{'label': 'sync device'}
{'label': 'spending history'}
{'label': 'schedule maintenance'}
{'label': 'spelling'}
{'label': 'payday'}
{'label': 

TypeError: ignored